In [2]:
import or_gym
import numpy as np
import gym
from gym import spaces
import matplotlib.pyplot as plt
import ray
from ray import tune
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import pandas as pd
from or_gym.envs.supply_network.multi_agent_inventory_management import MultiAgentNetInvMgmtEnv

In [5]:
class MAENVWrapper(MultiAgentNetInvMgmtEnv, MultiAgentEnv):
    
    def __init__(self, *args, **kwargs):
        super().__init__(self, *args, **kwargs)
        
def env_creator(*args, **kwargs):
    return MAENVWrapper()

In [7]:
env = MAENVWrapper()

def policy_gen(env, agent_id):
    return (None, env.observation_space, env.action_space[agent_id], {})

policy_graphs = {f'agent-{i}': policy_gen(i) for i in env.agent_ids}

def policy_mapping_fn(agent_id):
    return f'agent-{agent_id}'

env_name = "MANetOp"
tune.register_env(env_name, env_creator)

AttributeError: 'MAENVWrapper' object has no attribute 'agent_ids'

In [ ]:
config={
    "log_level": "WARN",
    "num_workers": 3,
    "num_cpus_for_driver": 1,
    "num_cpus_per_worker": 1,
    "lr": 5e-3,
    "model":{"fcnet_hiddens": [128, 128]},
    "multiagent": {
        "policies": policy_graphs,
        "policy_mapping_fn": policy_mapping_fn,
    },
    "env": env_name
}

exp_name = 'MANetOp-exp'

exp_dict = {
        'name': exp_name,
        'run_or_experiment': 'PPO',
        "stop": {
            "training_iteration": 100
        },
        'checkpoint_freq': 20,
        "config": config,
}
ray.init(ignore_reinit_error=True)
results = tune.run(**exp_dict)